In [ ]:
import pymysql
import pandas as pd

In [ ]:
conn = pymysql.connect(user='root', password='', database='sql_final')

In [ ]:
cursor = conn.cursor()

cursor.execute("SHOW TABLES")

records = cursor.fetchall()



In [ ]:
print(records)

In [ ]:
df = pd.read_csv('./NPA_TMA1_Need.csv')


In [ ]:
df

In [ ]:
for i, row in df.iterrows() :
    datetime = str(int(row['發生日期']))
    year = int(row['發生年度'])
    month = int(row['發生月份'])
    day = int(datetime[-2:])

    data = (datetime, year, month, day)
    query = "SELECT * FROM date WHERE datetime = %s "
    cursor.execute(query, (datetime))
    result = cursor.fetchone()

    # 如果資料不存在，則插入資料
    if not result:
        query = "INSERT INTO date (datetime, year, month, day) VALUES (%s, %s, %s, %s)"
        print(data)
        cursor.execute(query, data)
        print("insert:",datetime, year, month, day)

# 提交事務
conn.commit()

In [ ]:
for i, row in df.iterrows() :
    weather = str(row['天候名稱'])
    road_type = str(row['道路類別-第1當事者-名稱'])
    
    data= (weather,road_type)
    
    query = "SELECT * FROM condi WHERE (weather, road_type) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO condi (weather, road_type) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",weather, road_type)

# 提交事務
conn.commit()

In [ ]:
for i, row in df.iterrows() :
    cause_type = str(row['肇因研判子類別名稱-主要'])
    hit_and_run = str(row['肇事逃逸類別名稱-是否肇逃'])
    
    if hit_and_run == "否":
        hit_and_run=0
    elif hit_and_run == "是":
        hit_and_run=1
    
    data= (cause_type, hit_and_run)
    
    query = "SELECT * FROM cause WHERE (cause_type, hit_and_run) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO cause (cause_type, hit_and_run) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",cause_type, hit_and_run)

# 提交事務
conn.commit()

In [ ]:
for i, row in df.iterrows() :
    city = str(row['發生地點'])
    station = str(row['處理單位名稱警局層'])
    city=city[0:3]
    
    data= (city, station)
    
    query = "SELECT * FROM handle WHERE (city, station) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO handle (city, station) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:", city, station)

# 提交事務
conn.commit()

In [ ]:
cursor.close()

In [ ]:
conn.close()